In [1]:
import pandas as pd


In [4]:
data = pd.read_csv(
    "/Users/blue/Github/reco-ml-models/topk-off-policy-correction/DBRL/resources/tianchi.csv",
    sep=",",
    names=["user", "item", "label", "time", "sex", "age", "pur_power", "category", "shop", "brand"],
)[:100]

In [5]:
data

,user,item,label,time,sex,age,pur_power,category,shop,brand
0,322277,2028077,pv,242,0,1,3,14676,3959255,-1
1,628462,16644828,pv,318,0,1,9,7204,2357791,298554
2,1996053,18966363,pv,351,1,3,6,11215,76214,-1
3,1263779,3180376,pv,661,0,1,8,3439,2070152,64981
4,1425382,3282954,pv,703,1,1,4,8588,986630,380693
...,...,...,...,...,...,...,...,...,...,...
95,912986,22279738,pv,36444,1,2,9,439,4015932,99873
96,708006,25241126,pv,36790,1,1,3,7442,2037534,-1
97,309333,36507389,pv,37275,1,3,7,5119,2150973,581175
98,941909,2565628,pv,37387,1,2,4,8563,181652,46850


In [9]:
n_users = data.user.nunique()
n_users

77

In [29]:
user_indices = data.user.to_numpy()
user_indices = user_indices[:10]

In [30]:
import numpy as np

In [36]:
user_indices

array([ 322277,  628462, 1996053, 1263779, 1425382,  590414, 1067078,
       1269894, 1269894, 1269894])

In [37]:

users, user_position, user_counts = np.unique(
    user_indices,
    return_inverse=True,
    return_counts=True,
)

In [38]:
users

array([ 322277,  590414,  628462, 1067078, 1263779, 1269894, 1425382,
       1996053])

In [39]:
user_position

array([0, 2, 7, 4, 6, 1, 3, 5, 5, 5])

In [40]:
user_counts

array([1, 1, 1, 1, 1, 3, 1, 1])

In [42]:
user_split_indices = np.split(
    np.argsort(user_position),
    np.cumsum(user_counts)[:-1],
)
user_split_indices

[array([0]),
 array([5]),
 array([1]),
 array([6]),
 array([3]),
 array([7, 8, 9]),
 array([4]),
 array([2])]

In [48]:
split_indices_all = [[], []]
n_users = len(users)

for u in range(n_users):
    u_data = user_split_indices[u]
    u_data_len = len(u_data)
    if u_data_len <= 2:
        split_indices_all[0].extend(u_data)
    else:
        split_indices_all[0].extend(list(u_data[:2]))
        split_indices_all[1].extend(list(u_data[2:]))

split_indices_all

[[0, 5, 1, 6, 3, 7, 8, 4, 2], [9]]

In [71]:
split_data_all = list(data.iloc[idx] for idx in split_indices_all)
split_data_all


[      user      item label  time  sex  age  pur_power  category     shop   
 0   322277   2028077    pv   242    0    1          3     14676  3959255  \
 5   590414  17311119    pv   944    1    2          1      8954  3346137   
 1   628462  16644828    pv   318    0    1          9      7204  2357791   
 6  1067078  18657472    pv  1083    0    1          8      3814  3751540   
 3  1263779   3180376    pv   661    0    1          8      3439  2070152   
 7  1269894  33771711    pv  1207    1    2          6      7022  2804824   
 8  1269894  15372456    pv  1231    1    2          6      7022   131814   
 4  1425382   3282954    pv   703    1    1          4      8588   986630   
 2  1996053  18966363    pv   351    1    3          6     11215    76214   
 
     brand  
 0      -1  
 5  345468  
 1  298554  
 6  559398  
 3   64981  
 7      -1  
 8  274025  
 4  380693  
 2      -1  ,
       user      item label  time  sex  age  pur_power  category     shop   
 9  1269894  1713799

In [72]:
train_data, test_data = split_data_all

In [73]:
train_data

,user,item,label,time,sex,age,pur_power,category,shop,brand
0,322277,2028077,pv,242,0,1,3,14676,3959255,-1
5,590414,17311119,pv,944,1,2,1,8954,3346137,345468
1,628462,16644828,pv,318,0,1,9,7204,2357791,298554
6,1067078,18657472,pv,1083,0,1,8,3814,3751540,559398
3,1263779,3180376,pv,661,0,1,8,3439,2070152,64981
7,1269894,33771711,pv,1207,1,2,6,7022,2804824,-1
8,1269894,15372456,pv,1231,1,2,6,7022,131814,274025
4,1425382,3282954,pv,703,1,1,4,8588,986630,380693
2,1996053,18966363,pv,351,1,3,6,11215,76214,-1


In [74]:
counts = train_data["user"].value_counts()
counts

user
1269894    2
322277     1
590414     1
628462     1
1067078    1
1263779    1
1425382    1
1996053    1
Name: count, dtype: int64

In [75]:
freq = counts.index.tolist()
freq

[1269894, 322277, 590414, 628462, 1067078, 1263779, 1425382, 1996053]

In [76]:
mapping = dict(zip(freq, range(len(freq))))
mapping

{1269894: 0,
 322277: 1,
 590414: 2,
 628462: 3,
 1067078: 4,
 1263779: 5,
 1425382: 6,
 1996053: 7}

In [77]:
train_data["user"] = train_data["user"].map(mapping)
train_data

/var/folders/kz/brn1x6ms34b0dsp2hwdjmks40000gn/T/ipykernel_21665/3356251971.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["user"] = train_data["user"].map(mapping)


,user,item,label,time,sex,age,pur_power,category,shop,brand
0,1,2028077,pv,242,0,1,3,14676,3959255,-1
5,2,17311119,pv,944,1,2,1,8954,3346137,345468
1,3,16644828,pv,318,0,1,9,7204,2357791,298554
6,4,18657472,pv,1083,0,1,8,3814,3751540,559398
3,5,3180376,pv,661,0,1,8,3439,2070152,64981
7,0,33771711,pv,1207,1,2,6,7022,2804824,-1
8,0,15372456,pv,1231,1,2,6,7022,131814,274025
4,6,3282954,pv,703,1,1,4,8588,986630,380693
2,7,18966363,pv,351,1,3,6,11215,76214,-1


In [78]:
train_data.groupby("user").apply(len).to_dict()

{0: 2, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1}

In [85]:
train_sess_times = train_data["time"].astype('int').to_numpy()
train_sess_times

array([ 242,  944,  318, 1083,  661, 1207, 1231,  703,  351])

In [83]:
train_users, train_user_position, train_user_counts = np.unique(
    train_data.user.to_numpy(),
    return_inverse=True,
    return_counts=True
)
train_user_split_indices = np.split(
    np.argsort(train_user_position, kind="mergesort"),
    np.cumsum(train_user_counts)[:-1]
)

In [100]:
train_user_split_indices
np.where(np.ediff1d(np.array([1207, 1231, 1334, 1699, 18880])) > 100)[0]

array([1, 2, 3])

In [94]:
train_sess_times[[5,6]]

array([1207, 1231])

In [ ]:
)